In [8]:
import numpy as np
import pandas as pd
import pydot
import matplotlib.pyplot as plt
import itertools as iter
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, UpSampling2D

from PIL import Image

%matplotlib inline

In [9]:
class AE():
    def __init__(self, optimizer, loss):
        self.p_matrix = np.load("5x5matrix.npz")["p_matrix"]
        self.q_matrix = np.load("5x5matrix.npz")["q_matrix"]
        self.optimizer = optimizer
        self.model = Sequential([
            tf.keras.layers.Dense(25, activation = tf.nn.relu, input_dim = 16),
            tf.keras.layers.Dense(25, activation = tf.nn.relu),
            tf.keras.layers.Dense(25, activation = tf.nn.sigmoid)
        ])
        self.model.compile(optimizer = optimizer,
                           loss = loss,
                           metrics = ['accuracy'])
    def fit(self, epochs, batch_size):
        self.model.fit(self.q_matrix, self.p_matrix,
                      epochs = epochs,
                      batch_size = batch_size,
                      validation_data = (self.q_matrix, self.p_matrix))
    
    def predict(self, test_img, threshold):
        preds = self.model.predict(test_img).reshape(4,4)
        preds_binary = preds.copy()
        preds_binary[preds_binary < threshold] = 0
        preds_binary[preds_binary >= threshold] = 1
        ncols = 3
        fig, axes = plt.subplots(nrows = 1, ncols = ncols, figsize = (14,14))
        axes[0].imshow(test_img.reshape(3,3), cmap = plt.cm.binary)
        axes[0].set_title("original image")
        axes[1].imshow(preds, cmap = plt.cm.binary)
        axes[1].set_title("predicted image")
        axes[2].imshow(preds_binary, cmap = plt.cm.binary)
        axes[2].set_title("predicted image with binalization")
    
    def predict_RGB(self, test_img, threshold):
        preds = np.empty((4,4,3), dtype='float32')
        preds_binary = np.empty((4,4,3), dtype='float32')
        for dim in range(test_img.shape[2]):
            preds[ : , : , dim] = self.model.predict(test_img[ : , : , dim].reshape(1,9)).reshape(4,4)
            preds_binary[ : , : , dim] = preds[ : , : , dim].copy()
        preds_binary[preds_binary < threshold] = 0
        preds_binary[preds_binary >= threshold] = 1
        ncols = 3
        fig, axes = plt.subplots(nrows = 1, ncols = ncols, figsize = (14,14))
        axes[0].imshow(test_img, cmap = plt.cm.binary)
        axes[0].set_title("original image")
        axes[1].imshow(preds, cmap = plt.cm.binary)
        axes[1].set_title("predicted image")
        axes[2].imshow(preds_binary, cmap = plt.cm.binary)
        axes[2].set_title("predicted image with binalization")
        
    def SR(self, input_img, thresold):
        # Preprocessing area
        x, y, z = input_img.shape # dimention x and y
        pad_list = [] # padding count x = 0, y = 1
        for i in [x, y]:
            if i % 3 == 0:
                pad_list.append(0)
            elif i % 3 == 1:
                pad_list.append(2)
            else:
                pad_list.append(1)
        
        temp = np.zeros((x + pad_list[0], y + pad_list[1], 3), dtype='float32') # new temporary array replaced with input_img
        temp[ 0 : x, 0 : y , 0 : z] = input_img
        input_img = temp
        m,n = int(input_img.shape[0] / 3), int(input_img.shape[1] / 3)
        output_img = np.empty((4 * m , 4 * n, 3), dtype='float32')
        print(pad_list[0], pad_list[1])
        
        # SR area
        for dim in [0,1,2]:
            for i, j in iter.product(range(n), range(m)):
                partial_img = input_img.copy()[ 3 * j : 3 * ( j + 1 ), 3 * i : 3 * ( i + 1), dim] 
                if np.all(partial_img < 0.1):
                    partial_img = np.zeros(16)
                else:
                    partial_img = self.model.predict( partial_img.reshape(1,9) )
                output_img[4 * j : 4 * (j + 1), 4 * i : 4 * ( i + 1), dim] = partial_img.reshape(4,4)
        
        max_val = output_img.max()
        output_img /= max_val
        output_img_binalized = output_img.copy()
        output_img_binalized[output_img_binalized >= thresold] = 1
        output_img_binalized[output_img_binalized < thresold] = 0

        fig, axes = plt.subplots(1,3, figsize = (16,16))
        axes[0].imshow(input_img, cmap = plt.cm.binary)
        axes[0].set_title("input image")
        axes[1].imshow(output_img, cmap = plt.cm.binary)
        axes[1].set_title("output image")
        axes[2].imshow(output_img_binalized, cmap = plt.cm.binary)
        axes[2].set_title("output image with biinalization")
        
        return output_img
    
    def SR2(self, file_path, mean = 0 , sd = 0.2):
        # Resizing area
        input_img = Image.open(file_path)
        img_width, img_height = input_img.size
        
        ratio = 3 / 4
        resized_width =  int(input_img.size[0] * ratio)
        resized_height = int(input_img.size[1] * ratio)
        resized_img = input_img.resize((resized_width, resized_height))
        resized_img = np.array(resized_img, "f") / 255
        resized_img = resized_img + np.random.normal(mean, sd, (resized_height, resized_width, 3))
        input_img = np.array(input_img, 'f') / 255
        
        # Preprocessing area
        x, y, z = resized_img.shape # dimention x and y
        pad_list = [] # padding count x = 0, y = 1
        for i in [x, y]:
            if i % 3 == 0:
                pad_list.append(0)
            elif i % 3 == 1:
                pad_list.append(2)
            else:
                pad_list.append(1)
        
        temp = np.zeros((x + pad_list[0], y + pad_list[1], 3), dtype='float32') # new temporary array replaced with input_img
        temp[ 0 : x, 0 : y , 0 : z] = resized_img
        resized_img = temp
        m,n = int(resized_img.shape[0] / 3), int(resized_img.shape[1] / 3)
        output_img = np.empty((4 * m , 4 * n, 3), dtype='float32')
        print(pad_list[0], pad_list[1])
        
        # SR area
        for dim in [0,1,2]:
            for i, j in iter.product(range(n), range(m)):
                partial_img = resized_img.copy()[ 3 * j : 3 * ( j + 1 ), 3 * i : 3 * ( i + 1), dim] 
                if np.all(partial_img < 0.1):
                    partial_img = np.zeros(16)
                else:
                    partial_img = self.model.predict( partial_img.reshape(1,9) )
                output_img[4 * j : 4 * (j + 1), 4 * i : 4 * ( i + 1), dim] = partial_img.reshape(4,4)
        
        max_val = output_img.max()
        output_img /= max_val

        fig, axes = plt.subplots(1,3, figsize = (12,12))
        axes[0].imshow(input_img, cmap = plt.cm.binary)
        axes[0].set_title("input image")
        axes[1].imshow(resized_img, cmap = plt.cm.binary)
        axes[1].set_title("resized image")
        axes[2].imshow(output_img, cmap = plt.cm.binary)
        axes[2].set_title("output image")
        
        return output_img
    
    def SR3(self, test_img):
        preds = self.model.predict(test_img).reshape(4,4)
        preds_binary = preds.copy()
        preds_binary[preds_binary < threshold] = 0
        preds_binary[preds_binary >= threshold] = 1
        print(preds, preds_binary)
        
        ncols = 5
        fig, axes = plt.subplots(nrows = 1, ncols = ncols, figsize = (14,14))
        bayes_image = np.empty(16)
        index_when_given_q = np.all(self.q_matrix == test_img.reshape(9,), axis = 1)
        p_given_q = self.p_matrix[index_when_given_q]
        print(p_given_q)
        for col in range(16):
            is_one_counter = 0
            target_list = [p_given_q[row][col] for row in range(p_given_q.shape[0])]
            total_length = len(target_list)
            for j in range(total_length):
                if target_list[j] == 1:
                    is_one_counter += 1
                prob = is_one_counter / total_length
                bayes_image[i] = prob
        
        axes[0].imshow(test_img.reshape(3,3), cmap = plt.cm.binary)
        axes[0].set_title("original image")
        axes[1].imshow(preds, cmap = plt.cm.binary)
        axes[1].set_title("predicted image")
        axes[2].imshow(preds_binary, cmap = plt.cm.binary)
        axes[2].set_title("predicted image with binalization")
        axes[3].imshow(bayes_image.reshape(4,4), cmap = plt.cm.binary)
        axes[3].set_title("p(P|Q) image based on bayes")
        bayes_image[ bayes_image < thresold ] = 0
        bayes_image[ bayes_image >= thresold ] = 1
        axes[4].imshow(bayes_image.reshape(4,4), cmap = plt.cm.binary)
        axes[4].set_title("p(P|Q) image based on bayes with binalization")

In [10]:
autoencoder = AE(optimizer = tf.train.AdamOptimizer(learning_rate=5e-4) , loss = tf.keras.losses.binary_crossentropy)

In [11]:
autoencoder.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 25)                425       
_________________________________________________________________
dense_10 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_11 (Dense)             (None, 25)                650       
Total params: 1,725
Trainable params: 1,725
Non-trainable params: 0
_________________________________________________________________


In [12]:
autoencoder.fit(epochs = 5, batch_size=1024)

Train on 33554432 samples, validate on 33554432 samples
Epoch 1/5
33554432/33554432 [==============================] - 438s 13us/step - loss: 0.2343 - acc: 0.8980 - val_loss: 0.2170 - val_acc: 0.9074
Epoch 2/5
33554432/33554432 [==============================] - 414s 12us/step - loss: 0.2170 - acc: 0.9074 - val_loss: 0.2170 - val_acc: 0.9074
Epoch 3/5
33548288/33554432 [============================>.] - ETA: 0s - loss: 0.2170 - acc: 0.9074

KeyboardInterrupt: 